# COMP90014 Assignment 1
### Semester 2, 2024
Version 1.0 Last edited 30/07/2024.

In [42]:
### Fill in your student details here
NAME = "Ching-Yi Peng"
ID = "1545824"

## Completing the assignment

***Academic integrity***

This assignment should be completed by each student individually. <br>
Make sure you read this entire document, and ask for help if anything is not clear. <br>
Any changes or clarifications to this document will be announced via the LMS. <br>

Please make sure you review the University's rules on academic honesty and plagiarism: https://academichonesty.unimelb.edu.au/

Do not copy any code from other students or from the internet. This is considered plagiarism.

***Completing the assignment & submission***

To complete the assignment, finish the coding and short answer tasks in this notebook.<br>
Please do not copy or delete cells in this notebook, as this may interrupt the autograding of hidden tests. <br>
Your completed notebook file containing all your answers will be turned in via LMS. Please also submit an HTML file.

***Visible & hidden tests***

In some cases, we have provided test input and test output that you can use to try out your solutions. <br> 
These are visible tests and serve to warn you if you've made a mistake but are **not** exhaustive.

During assessment, there are also hidden tests we run to validate your code. <br>
As you won't see the hidden tests, it's up to you to decide whether your code is correct.

**Remember to save your work early and often.**

## Marking

***Graded cells***

Cells that must be completed to receive marks are clearly labelled with the following text:

`# -- GRADED CELL (N marks) - complete this cell --`

Only add answers to these cells. 

Some cells are code cells, in which you must complete the code to solve a problem. <br>
Others are markdown cells, in which you must write your answers to short-answer questions. 

<br>

***Completing code cells***

You will see the following text in graded code cells.

``` python
raise NotImplementedError()
```

You must remove the `raise NotImplementedError()` line from the cell, and replace it with your solution. <br>
If you want to import a library or use a helper function, the import statements must be inside the function for that question. <br>
For example:
``` python
# -- GRADED CELL (1 mark) - complete this cell --

def build_simple_index(seq: str, k: int) -> dict:
    from collections import defaultdict
```

Include code comments in your solutions! <br>
Well commented code can help you to receive partial marks even if the final solution is incorrect. 

<br>

***Editing the notebook***

Only graded cells will be marked.
- Do **NOT** enter solutions outside of graded cells
- Do **NOT** duplicate or remove cells from the notebook
- You may add new cells to test code, but new cells will not be graded.
- Word limits, where stated, will be strictly enforced. Answers exceeding the limit **will not be marked**.

<br>

***Marks***

The total marks for the assignment add up to 10. <br>
This assignment will be worth 10% of your overall subject grade.<br>
No marks are allocated to commenting in your code. <br>
We do however, encourage efficient and well commented code.



## Submission

Make sure you have filled in any place that says `# -- GRADED CELL (N marks) - complete this cell --`.

Before you turn this assignment in, make sure everything runs as expected, and the output is cleared. 

First, **restart the kernel**:

- In the menubar, select Kernel -> Restart.

Next, **run all cells**:

- In the menubar, select Cell -> Run All.

Finally, **clear all output**:

- In the menubar, select Options -> Clear All Outputs

Your completed notebook file containing all your answers must be turned in via LMS in `.ipynb` format. <br>
You must also submit a copy of this notebook in `html` format with the output cleared.

Your submission should include **only two** files with names formatted as: **Assignment1.ipynb** and **Assignment1.html**



<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    

<h1>ASSIGNMENT 1</h1>

<h2>Using BLAST to Identify Plastic-Degrading Genes</h2>

</div>

## Introduction 

For this assignment, pretend you have sequenced a previously undocumented bacteria.  
The bacteria was isolated from a rubbish tip which contains large amounts of different plastics. 
You are curious know whether this unknown organism may have the ability to break down plastic. 

To test, you have sequenced & assembed the organism's genome, identified open-reading-frames (ORFs), and have translated each ORF into an amino acid sequence. 
This collection of amino acid sequences reflects the proteome of your unknown bacteria.
You now plan to answer your research question by comparing the proteome to a database of previously known plastic-degrading proteins, to see if there are any matches. 

In this assignment we will explore strategies used by BLAST to match proteome proteins to the plastic database.
Questions 1-5 provide a toolkit of techniques which you can use in Question 6 to identify plastic-degrading genes. 

- Q1: Build (& query) a k-mer index using *a single* sequence
- Q2: Build (& query) a k-mer index using *multiple* sequences 
- Q3: Linking matched k-mers 
- Q4: Ungapped alignment 
- Q5: Gapped alignment 
- Q6: Identify plastic-degrading genes

The final output of our code from Question 6 will be a list of tuples, where each tuple represents a sequence match between a protein from the 'unknown' bacteria, and a protein from the database. 
For example, if the protein named <small>`WP_003141008.1`</small> from our 'unknown' bacteria is similar to the <small>`00148||Protease||Amycolatopsis_orientalis||PLA`</small> protein in the plastic database, our output to the final question should contain the following tuple: 
<small>`('WP_003141008.1', '00148||Protease||Amycolatopsis_orientalis||PLA')`</small>

Question 6 is a special question. You are free to implement Q6 however you wish, as long as indexing is used in some manner. 
You may call your previous functions from Q1-5, write new implementations for these functions, or rip it all up and use a custom strategy of your own. 
Points are awarded for both correctness ***and*** runtime performance in question 6, so you are incentivised to play around with your approach to find the most efficient strategy. 


## The Data

To begin, let's have a look at the data we will be working with. 

Create a folder called 'data' in your working directory, then download both fasta files from the url below, and place them in the 'data' folder.<br>
You can manually download these two files using a web browser, or use wget/curl on the command line. 

https://github.com/melbournebioinformatics/COMP90014_2024/tree/master/assignments/data. 

Our unknown organism ***proteome*** is read from a FASTA file (<small>`data/unknown.fasta`</small>) containing many protein sequences. <br>
Each entry has a name, followed by the sequence.
```
>WP_000995360.1
MNAYTVSRLALDAGVSVHIVRDYLLRGLLRPVACTPGGYGLFDDAALQRLCFVRAAFEAGIGLDALARLCRALDAADGDE
AAAQLALLRQFVERRREALADLEVQLATLPTEPAQHAESLP
```

Our PlasticDB protein ***database*** is also read from a FASTA file (<small>`data/PlasticDB.fasta`</small>) containing many protein sequences. <br>
The entry names are structured as <small>`> id || gene name || species || gene symbol`</small>.

```
>00001||PHB_depolymerase||Ralstonia_pickettii||PHB_PHA
MKHPYGYRWHWLYALVVTLMTALATFSAHAAVTAGPGAWSSQQTWAADTVNGGNLTGYFYWPASQPTTPNGKRALVLVLH
GCLQTASGDVIDNANGAGFNWKTIAEQYGAVVLAPNATGNVYSNHCWDYANTSPSRTSGHVGVLLDLVNRFVTNSQYAID
PNQVYVAGLSSGGGMTMVLGCIAPDIFAGIGINAGPPPAITRWKIGVVPSGYTATTAANNCKAWAGSNASSFNTQIAGAV
WGTSDYTVAQAYGPMDTAAFRQIYGGTFTQGAQVSISGGGTNTPYTDSNGKLRTHEISVSGMAHAWPAGTGGDNNNYVDA
THINYPAFVMDYWVKNNLRAGSGPVQSAGTPTGLTVTGTTTTSVSLSWNAVTNATSYNVYRNGSKVGSSTSTTYTDTGLI
AGTTYSYTVTEIDPTAGESAQSSAVSAKTQSSFACTATTASNYAHVQAGRAHDSGGIAYANGSNQSMGLDNVFYTNTLAQ
TAAGYYVIGNCP
```

We will be using python tuples for any in-memory representation of these proteins. <br>
Each protein will be formatted as a python tuple with the format (name, sequence). <br>
Our datastructure is therefore a list of tuples <small>`list[Tuple[str, str]]`</small>, where each tuple represents a protein. 


## Type hints

This assignment makes use of python type hints. 

The role of type hints is to tell the programmer (and the python interpreter) what datatype to expect for a given variable. <br>
Python treats them similar to code comments, so they don't actually affect execution. <br>
If you tell python that a variable called <small>`my_int`</small> is an integer, but actually pass a string, python will not throw an error. 

Run the cell below so that python understands the type hints in this assignment. 


In [43]:
from typing import Tuple, Any


## Completing Questions

**Keep it simple.**

As a personal opinion, your first attempt to solve each question should be as simple as possible. <br>
Your first priority is correct output, not efficiency. <br>
Brute force implementations are 100% fine and will help you understand the question better. <br>
If you have a working solution, you can optimise it later (in Question 6). 

**If having trouble with a question, move to the next one.**

This may be relevant for Question 3, which can be tricky to get right.
Question 6 may also pose a challenge. 

<br>



<div style="color: rgb(0,0,0); background: #0096FF; border: solid 1px rgb(0,0,0); padding: 10px;">

<h2>Section A: K-mer indexes</h2>

</div>

### Background

An important strategy used by BLAST is construction of a k-mer index. 
This index allows fast identification of exact subsequence (k-mer) matches between a query sequence, and the reference sequence(s) used to build the index. 
By scanning along the query sequence & extracting k-mers, we can do index lookups to find whether the k-mer was also present in the reference sequence(s).

Position information of these k-mer matches can also be recovered if we:
- [ref]   storing a list of positions for each ref k-mer
- [query] keeping track of our position (as we extract each query k-mer)

For our task, our k-mer index will be built from the PlasticDB protein sequences.
This will enable us to rapidly identify identical subsequences in query proteins from the unknown proteome. 

There are many proteins in the PlasticDB database, so our index will eventually need to handle multiple sequences.
We will start with a simplified problem, where we build a k-mer index from a *single* input protein sequence. 
We will then progress to index building (and query) using multiple protein sequences. 

<br>




<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1a</h3>

To start, let's build an index for a single protein in the PlasticDB database. 

<b>Challenge:</b> Write a function which generates a k-mer index from a single input sequence. 
- [ ] Input: A sequence (str); and k-mer size k (int)
- [ ] Ouput: A python dictionary containing kmers as keys, and the locations they appear in the sequence (list of integers) as values. 
- [ ] Return an empty dictionary if inputs do not yield any kmers.

Notes: 
- For each location in the sequence, you will need to extract a kmer of size k, then update your dictionary for that kmer and its new witnessed location. 
- The location of the first kmer you extract is loc = 0 (same as list indexing).  
- Ensure you check the value of k is valid for the sequences. 
    
</div>

Return an empty dictionary if inputs do not yield any kmers.

Notes: 
- For each location in the sequence, you will need to extract a kmer of size k, then update your dictionary for that kmer and its new witnessed location. 
- The location of the first kmer you extract is loc = 0 (same as list indexing).  
- Ensure you check the value of k is valid for the sequences. 
    
</div>

In [44]:
# -- GRADED CELL (1 mark) - complete this cell --

def build_simple_index(seq: str, k: int) -> dict:
    
    if k <= 0 or k > len(seq):
        return {}
    
    kmer_index = {}
    for i in range (len(seq) - k + 1):
        kmer = seq[i : i + k]
        if kmer in kmer_index:
            kmer_index[kmer].append(i)
        else:
            kmer_index[kmer] = [i]
        
    return kmer_index    

In [45]:
# extra code cell for development if needed
sequence = "ATGCGATGATCG"
k = 3
index = build_simple_index (sequence, k)
print(index)

{'ATG': [0, 5], 'TGC': [1], 'GCG': [2], 'CGA': [3], 'GAT': [4, 7], 'TGA': [6], 'ATC': [8], 'TCG': [9]}


In [46]:
# Testing cell - Do not alter.

print("--- test ---")
the_index = build_simple_index("MSTLSWTLSVR", 3)
print('EXPECTED')
print('\n'.join(['MST [0]','STL [1]','TLS [2, 6]','LSW [3]','SWT [4]','WTL [5]','LSV [7]','SVR [8]']))
print('\nACTUAL')
for kmer, locs in the_index.items():
    print(kmer, locs)


--- test ---
EXPECTED
MST [0]
STL [1]
TLS [2, 6]
LSW [3]
SWT [4]
WTL [5]
LSV [7]
SVR [8]

ACTUAL
MST [0]
STL [1]
TLS [2, 6]
LSW [3]
SWT [4]
WTL [5]
LSV [7]
SVR [8]


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 1b</h3>

Let's use our k-mer index to identify matching subsequences in a given query.

<b>Challenge:</b> Write a function which finds ***all*** matching k-mers between a query sequence and an index (built from a reference sequence). 
- [ ] Input: A query sequence (str); a k-mer size k (int); a k-mer index (dict)
- [ ] Ouput: A list of tuples representing k-mer matches, sorted by query sequence position (second element in tuple).
- [ ] Return an empty list if no matching k-mers. 

The output tuples should have 3 elements each. 
1. the k-mer
2. position in query sequence
3. position in reference sequence

For each matching k-mer, ensure all possible location combinations appear in the output. 
For example, if a given k-mer appears once in the query and twice in the reference, this should produce 2 tuples:
- (kmer, query_pos, ref_pos_1)
- (kmer, query_pos, ref_pos_2)

    
</div>

In [47]:
# -- GRADED CELL (1.5 marks) - complete this cell --

def get_index_hits(query: str, k: int, the_index: dict) -> list[Tuple]:
    
    if k <= 0 or k > len(query):
        return []
    
    matching_kmer = []
    for i in range (len(query) - k + 1):
        kmer = query[i : i + k]
        if kmer in the_index:
            for ref_pos in the_index[kmer]:
                matching_kmer.append((kmer, i, ref_pos))
                
    matching_kmer.sort(key = lambda x: x[1])
    return matching_kmer        

In [48]:
# extra code cell for development if needed

In [49]:
# Testing cell - Do not alter.

the_index = {'MS': [0],'ST': [1],'TL': [2],'LS': [3, 7],'SW': [4],'WM': [5],'ML': [6],'SV': [8],'VR': [9]}
print("--- test ---")
print('EXPECTED')
print("""\
MS 0 0
ST 1 1
TL 5 2
LS 6 3
LS 6 7
ST 7 1
""")

print('ACTUAL')
for kmer, pos1, pos2 in get_index_hits("MSTAWTLST", 2, the_index):
    print(kmer, pos1, pos2)


--- test ---
EXPECTED
MS 0 0
ST 1 1
TL 5 2
LS 6 3
LS 6 7
ST 7 1

ACTUAL
MS 0 0
ST 1 1
TL 5 2
LS 6 3
LS 6 7
ST 7 1


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 2</h3>

Now that we have built & queried an index using a single sequence as input, let's progress to handling multiple protein sequences.

Your k-mer index can have any structure you like. <br>
As the implementation details are left to you, we don't know the structure of your index. 
For this reason, you will write two functions in this question:
- <small>`build_index()`</small> which generates a k-mer index from multiple protein sequences
- <small>`query_index()`</small> which queries your index for a given k-mer.
    
<b>Challenge 1:</b> Write a function called <small>`build_index()`</small> which builds a k-mer index from multiple input sequences.
- [ ] Input: A list of (protein name, sequence) tuples (list[Tuple]); and a k-mer size k (int)
- [ ] Ouput: A single object which represents your index. 

Return a single object which contains your index. <br>
This could be a single flat dictionary, a nested dictionary, a list containing multiple dictionaries, a list of strings and a dictionary etc. 
This single object must be all that is required to pass to <small>`query_index()`</small> for proper function (aside from the k-mer to query). 

<b>Challenge 2:</b> Write a function called <small>`query_index()`</small> which queries your k-mer index.
- [ ] Input: A query kmer (str); a k-mer index (Any)
- [ ] Ouput: A list of (sequence name, list[location]) tuples.

Your index will be built from multiple sequences. Therefore, the <small>`query_index()`</small> function can't return a simple list of integer locations where the k-mer 'appears'.   
Instead, we need to return locations grouped by the sequence they're from. See the test case for an example.

</div>

In [50]:
# -- GRADED CELL (1.5 marks) - complete this cell --

def build_index(sequences: list[Tuple], k: int) -> Any:
    
    index = {}
    for pro_nam, seq in sequences:
        for i in range (len(seq) - k + 1):
            kmer = seq[i : i + k]
            if kmer not in index:
                index[kmer] = []
            index[kmer].append((pro_nam, i))
    return index

def query_index(kmer: str, the_index: Any) -> list[Tuple]:
    
    results = []
    if kmer in the_index:
        pro_mapping = {}
        for pro_nam, position in the_index[kmer]:
            if pro_nam not in pro_mapping:
                pro_mapping[pro_nam] = []
            pro_mapping[pro_nam].append(position)
        
        for pro_nam, positions in pro_mapping.items():
            results.append((pro_nam, positions))
    return results

In [51]:
# extra code cell for development if needed

In [52]:
# Testing cell - Do not alter.

print('\n--- test ---')
K = 3
QUERY_KMER = 'YYY'
REF_PROTEINS = [
    ('>protein 1', 'MKHPYYYRW'), 
    ('>protein 2', 'MNYYYKNLYYYAT')
]
the_index = build_index(REF_PROTEINS, K)
actual_hits = query_index(QUERY_KMER, the_index)
expected_hits = [('>protein 1', [4]), ('>protein 2', [2, 8])]
print('\nexpected ---')
print('name\t\tlocations')
for seqname, locs in expected_hits:
    print(str(seqname) + '\t' + str(locs))
print('\nactual ---')
print('name\t\tlocations')
for seqname, locs in actual_hits:
    print(str(seqname) + '\t' + str(locs))



--- test ---

expected ---
name		locations
>protein 1	[4]
>protein 2	[2, 8]

actual ---
name		locations
>protein 1	[4]
>protein 2	[2, 8]



<div style="color: rgb(0,0,0); background: #0096FF; border: solid 1px rgb(0,0,0); padding: 10px;">

<h2>Section B: High-Scoring Segment Pairs (HSPs)</h2>

</div>

### Background

Rather than directly using k-mer matches as seed locations for pairwise alignment, BLAST performs another heuristic step to further refine its list of candidate locations. 
This next step uses k-mer hits to identify 'high-scoring segment pairs' (HSPs), which are larger regions of similarity between the query and reference sequence. 
The span of this region must be equal in both sequences, but can be any arbitrary size (ie 10bp, 100bp etc).

HSPs are identified between a pair of sequences using the following broad steps: 
1. Identify short matching subsequences (k-mers).
2. Link k-mer matches to form a chain if they're colinear (same relative spacing in both sequences) and close to each other. 
3. Fill gaps & extend the segment ends to form a HSP. 

Regarding step (2) above, the linked k-mers form the initial backbone of a HSP.  <br>
The rules for this linking process in BLAST are relatively complex.  <br>
For this reason we will simplify step 2 by changing the rules:
- We will only link 2 kmers.
- If there are multiple valid chains, we will return the leftmost (earliest in sequence) from the perspective of the query.
- If there are multiple kmers we can link from a given starting kmer, we will link the closest.

See the figure below for a diagram of two linked k-mer matches forming a chain.

<img src="https://raw.githubusercontent.com/melbournebioinformatics/COMP90014_2024/master/assignments/media/hsp.png" width="700">

<br>


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 3</h3>

<b>Challenge:</b> Write a function which queries your k-mer index to identify & link a valid chain of two k-mer matches. 
- [ ] Input: A query sequence (str); a k-mer size k (int); a k-kmer index (dict); the maximum spacing between k-mers (int). 
- [ ] Output: A k-mer chain represented as a tuple of integers (Tuple[int, int, int]).
- [ ] Return None if there are no valid chains. 

The output tuple should contain three items:
- The first item encodes the start position of kmer1 in the query sequence. 
- The second item encodes the start position of kmer1 in the reference sequence (obtained using your k-mer index). 
- The third item is the spacing distance between kmer1 & kmer2. 

Additional instructions: 
- Only link 2 k-mer matches to form a chain.
- The spacing between kmer1 and kmer2 must be ***identical*** in both the query sequence and the index (built from reference sequence). 
- The spans of kmer1 and kmer2 cannot overlap.
- If there are multiple valid chains, return the leftmost (earliest in sequence) from the perspective of the query.
- If there are multiple linkable k-mers from a given starting k-mer, link the closest.
    
</div>

In [53]:
# -- GRADED CELL (1 mark) - complete this cell --

def link_kmers(query: str, k: int, the_index: dict, max_spacing: int) -> Tuple[int, int, int] | None:
    best_result = None
    
    # Loop through each possible starting position of a k-mer in the query sequence
    for i in range(len(query) - k + 1):
        kmer1 = query[i:i+k]
        
        # Check if the current k-mer is in the index
        if kmer1 in the_index:
            # Get all starting positions of this k-mer in the reference sequence
            ref_positions = the_index[kmer1]
            
            # Now check for a second k-mer within the allowed spacing
            for j in range(i + k, min(i + k + max_spacing + 1, len(query) - k + 1)):
                kmer2 = query[j:j+k]
                
                # Check if the second k-mer is the same as the first and is in the index
                if kmer2 in the_index:
                    
                    # Check all positions of the second k-mer in the reference sequence
                    for ref_pos1 in ref_positions:
                        distance_kmer = j - i
                        expected_ref_pos2 = ref_pos1 + distance_kmer
                        spacing = distance_kmer - k
                        
                        # Check if the expected position is in the actual positions of the second k-mer
                        if expected_ref_pos2 in the_index[kmer2]:
                            # Select the smallest starting index and corresponding reference position
                            if best_result is None or (i < best_result[0]) or (i == best_result[0] and ref_pos1 < best_result[1]):
                                best_result = (i, ref_pos1, spacing)

    return best_result

In [54]:
# extra code cell for development if needed

In [55]:
# Testing cell - Do not alter.

K = 3
MAX_SPACING = 5

# test1: only 1 valid chain ---
the_index = {'ACC': [0], 'CCA': [1], 'CAT': [2], 'ATA': [3], 'TAG': [4, 9], 'AGC': [5], 'GCG': [6], 'CGT': [7], 'GTA': [8], 'AGT': [10], 'GTG': [11], 'TGC': [12], 'GCA': [13], 'CAG': [14], 'AGA': [15], 'GAA': [16]}
query = "TACGTAGAGAAGTCGATGC"
chain = link_kmers(query, K, the_index, MAX_SPACING)
print(f'[Test 1]: Expected=(4, 4, 3) Actual={chain}')

# test2: multiple valid spacings ---
the_index = {'ACC': [0], 'CCA': [1, 13], 'CAT': [2], 'ATA': [3], 'TAG': [4, 9], 'AGC': [5], 'GCG': [6], 'CGT': [7], 'GTA': [8], 'AGT': [10], 'GTC': [11], 'TCC': [12], 'CAG': [14], 'AGA': [15], 'GAA': [16]}
query = "TACGTAGAGAAGTCGATGC"
chain = link_kmers(query, K, the_index, MAX_SPACING)
print(f'[Test 2]: Expected=(4, 4, 3) Actual={chain}')

# test3: multiple valid chains ---
the_index = {'ACC': [0], 'CCA': [1], 'CAT': [2], 'ATA': [3], 'TAG': [4, 7, 12], 'AGT': [5, 13], 'GTA': [6, 11], 'AGC': [8], 'GCG': [9], 'CGT': [10], 'GTG': [14], 'TGT': [15], 'GTT': [16], 'TTT': [17], 'TTC': [18], 'TCA': [19]}
query = "TACGTAGAGAAGTCTTTGA"
chain = link_kmers(query, K, the_index, MAX_SPACING)
print(f'[Test 3]: Expected=(3, 6, 4) Actual={chain}')



[Test 1]: Expected=(4, 4, 3) Actual=(4, 4, 3)
[Test 2]: Expected=(4, 4, 3) Actual=(4, 4, 3)
[Test 3]: Expected=(3, 6, 4) Actual=(3, 6, 4)


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 4</h3>

Now that we have a chain of 2 k-mer matches, we will use ungapped alignment to fill the gap and extend in either direction to form a HSP. 

Here we will refer to the query sequence as 'seq1', and the reference sequence (used to build k-mer index) as 'seq2'. <br>
Note that we will not be using a k-mer index, as need to directly compare seq1 and seq2.
    
<b>Challenge:</b> Write a function which fills & extends a k-kmer chain. 
- [ ] Input: seq1 (str); seq2 (str); a tuple representing a k-mer chain (Tuple[int, int, int]); a k-mer size k (int); max_mismatches (int)
- [ ] Output: A tuple representing the HSP span (Tuple[int, int, int])
- [ ] Return None if filling the gap exceeds max_mismatches.

Keep in mind that the width of the output HSP is identical in both sequences. Conversely, the relative start location of the HSP can differ in seq1 vs seq2. <br>
The output tuple should therefore contain three items:
- The HSP start position in seq1.
- The HSP start position in seq2.
- The width of the HSP.

Additional instructions:
- Fill the gap between k-mer matches first.
- If filling the gap exceeds max_mismatches, return None.
- The number of mismatches encountered during gap filling should deduct from max_mismatches. 
- Use hamming distance to extend the HSP on either end, until max_mismatches (reduced).
- Mismatches encountered during extension of one HSP end ***does not*** alter max_mismatches for the other HSP end.

Note: a value for k-mer size (k) is required to identify the gap section from both sequences, and the downstream k-mer end. 
</div>

In [56]:
# -- GRADED CELL (1 mark) - complete this cell --
def ungapped_extension(seq1: str, seq2: str, chain: Tuple, k: int, max_mismatches: int) -> Tuple[int, int, int] | None:
    kmer1_start1, kmer1_start2, spacing = chain
    kmer2_start1 = kmer1_start1 + k + spacing
    kmer2_start2 = kmer1_start2 + k + spacing

    # Handling the gap mismatches first
    gap_seq1 = seq1[kmer1_start1 + k:kmer2_start1]
    gap_seq2 = seq2[kmer1_start2 + k:kmer2_start2]
    mismatches = sum(1 for a, b in zip(gap_seq1, gap_seq2) if a != b)

    if mismatches > max_mismatches:
        return None

    remaining_mismatches = max_mismatches - mismatches

    # Extend left from the start of the first k-mer
    extended_start1, extended_start2 = kmer1_start1, kmer1_start2
    while extended_start1 > 0 and extended_start2 > 0:
        extended_start1 -= 1
        extended_start2 -= 1
        if seq1[extended_start1] != seq2[extended_start2]:
            if remaining_mismatches == 0:
                extended_start1 += 1
                extended_start2 += 1
                break
            remaining_mismatches -= 1

    # Extend right from the end of the second k-mer
    extended_end1, extended_end2 = kmer2_start1 + k - 1, kmer2_start2 + k - 1  # up to the last character of the second k-mer
    while extended_end1 < len(seq1) and extended_end2 < len(seq2):
        if seq1[extended_end1] != seq2[extended_end2]:
            if remaining_mismatches == 0:
                break
            remaining_mismatches -= 1
        extended_end1 += 1
        extended_end2 += 1

    hsp_length = extended_end1 - extended_start1 # Calculating the length including both ends
    return (extended_start1, extended_start2, hsp_length)

In [57]:
# extra code cell for development if needed

In [58]:
# Testing cell - Do not alter.

# test1: should extend right hand side by 2 ---
seq1 = '----KMER1---KMER2-----'
seq2 = '++++KMER1---KMER2--++-'
chain = (4, 4, 3)
actual = ungapped_extension(seq1, seq2, chain, k=5, max_mismatches=0)
print(f'test1: expected=(4, 4, 15)  actual={actual}')

# test2: mismatch in gap, sequences are offset ---
seq1 = '----KMER1---KMER2-----'
seq2 = 'x++++KMER1-+-KMER2--++-'
chain = (4, 5, 3)
actual = ungapped_extension(seq1, seq2, chain, k=5, max_mismatches=2)
print(f'test2: expected=(3, 4, 17)  actual={actual}')



test1: expected=(4, 4, 15)  actual=(4, 4, 15)
test2: expected=(3, 4, 17)  actual=(3, 4, 16)


<div style="color: rgb(0,0,0); background: #0096FF; border: solid 1px rgb(0,0,0); padding: 10px;">

<h2>Section C: Final Alignment & Identifying Plastic Degrading Proteins</h2>

</div>

### Background

The process thus far aims to vastly reduce the number of candidate locations for a final pairwise alignment between two sequences. <br>
The HSPs produced in question 4 represent these candidate regions.

Up to this point we have:
- Indexed a reference sequence using k-mers
- Extracted k-mers from a query sequence to find exact subsequence matches
- Linked these exact matches into a chain
- Filled the gap & extended the ends to finalise our HSP. 

The final, comprehensive assessment of similarity between the two sequences will be to perform pairwise alignment using these candidate regions. 

***Wait, so what was all this for??***

Keep in mind that we have only been comparing a ***single*** query sequence to a ***single*** reference sequence.  <br>
Our ultimate goal is a bit different. For a given protein in our 'unknown' organism, we want to find whether there is similarity to ***any*** of the PlasticDB database proteins. <br>
Since there are 219 protein sequences in the PlasticDB database, for each 'unknown' protein we need to check all 219 PlasticDB proteins for similarity. <br>
Rather than doing 219 pairwise alignments for a given 'unknown' protein (inefficient), the steps above reduce this to just a few (if any) candidate locations to assess.

***The final 2 questions***

*Question 5* 

Question 5 implements the final assessment of similarity between two sequences - pairwise alignment. 
Using a HSP as a candidate region, we will perform local (smith-waterman) alignment to produce a final assessment of similarity. 
We will use local alignment (rather than global alignment) because this is what BLAST uses (and for good reason!). 

*Question 6* 

Here we will put everything together. All sequences from the 'unknown' organism proteome will be compared to PlasticDB proteins to find sequence similarity. <br>
We will:
- Read the PlasticDB proteins from file to create a k-mer index from all sequences (reference).
- Read the 'unknown' organism proteins from file.
- For each query protein (from the 'unknown' organism), assess whether it has similarity to any PlasticDB proteins. 

You are free to implement Question 6 however you like (as long as indexing is used in some form!).
Questions 1-5 are a toolkit of strategies which you can use in Question 6. 
You are encouraged to play around with your functions & overall strategy in question 6 to optimise efficiency.
Extra points are awarded for fast execution (on a standard test computer).

<br>



<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 5</h3>

Up to this point we have not allowed gaps (caused by indels) between any two sequences. <br>
Here we perform a final pairwise alignment, which allows gaps.

<b>Challenge:</b> Write a function which performs local alignment between two sequences. 
- [ ] Input: seq1 (str), seq2 (str), a HSP (Tuple[int, int, int]), min_score (int) 
- [ ] Output: a Bio.PairwiseAlignment object
- [ ] Return None if the alignment score is less than min_score.

Use the Bio.Align.PairwiseAligner class from Biopython to perform alignment. <br>
See the cell below for an example of using PairwiseAligner. 
- Call the .align() function to do alignment.
- The .align() function returns a list of alignments, where the first item is the best alignment.
- The score for an alignment can be retrieved via the .score property.
- Documentation can be seen at https://biopython.org/docs/dev/api/Bio.Align.html if needed. 

<br>

***Note*** 

The challenge of this question is to properly define the alignment window of the two sequences.

Keep in mind that <small>`seq1`</small> can be much smaller than <small>`seq2`</small>. 
We wish to use *local alignment* rather than *semi-global alignment*, so cannot just provide the full sequence of seq2. <br>
Therefore we should align ***all*** of <small>`seq1`</small> to a reasonable ***section*** of <small>`seq2`</small>. <br>
This section of <small>`seq2`</small> can be identified using the HSP tuple, and the length of <small>`seq1`</small>. <br>
In the example below, we would select the section [10, 26] from <small>`seq2`</small> for alignment. 

```
# Info: The HSP tuple is comprised of (seq1_start, seq2_start, width)

HSP:  (4, 14, 9)
                      |- HSP -|
seq1:            CATS GGGGGGGAA ACGG
seq2: ATATATATAT TAGA GGGGGGGTA GCCG AATTAATATATATATATTATA
                 ^10             26^

```
<br>



</div>

In [59]:
# PairwiseAligner example

import Bio.Align
aligner = Bio.Align.PairwiseAligner()
aligner.mode = 'local'
alignments = aligner.align("GAACT", "GAT")
alignment = alignments[0]
print(alignment)


ModuleNotFoundError: No module named 'Bio'

In [ ]:
# -- GRADED CELL (1 mark) - complete this cell --
import Bio.Align

def local_alignment(seq1: str, seq2: str, hsp: Tuple[int, int, int], min_score: int) -> Bio.Align.Alignment | None:
    aligner = Bio.Align.PairwiseAligner()
    aligner.mode = 'local'  # Change to global alignment

    # Extract start and width from the HSP tuple
    seq1_start, seq2_start, width = hsp
    
    # Calculate the end indices, ensuring we do not go out of bounds
    seq1_end = min(seq1_start + width, len(seq1))
    seq2_end = min(seq2_start + width, len(seq2))
    
    # Extract regions for alignment based on the HSP tuple
    seq1_region = seq1[seq1_start - min_score : seq1_end + min_score + 1]
    seq2_region = seq2[seq2_start - min_score : seq2_end + min_score + 1]

    # Perform alignment
    alignments = aligner.align(seq1_region, seq2_region)

    # Check for alignments and if the best score meets the minimum score requirement
    if alignments and alignments[0].score >= min_score:
        return alignments[0]
    else:
        return None
    

ModuleNotFoundError: No module named 'Bio'

In [ ]:
# extra code cell for development if needed


In [ ]:
# Testing cell - Do not alter.

# test1: basic case ---
hsp = (0, 10, 4)
seq1 = 'GGGG'
seq2 = 'ATATATATATGGGGAT'
print('\n--- test1 ---\n')
print('[expected]')
print("""\
target            0 GGGG 4
                  0 |||| 4
query             0 GGGG 4
""")
print('[actual]')
print(local_alignment(seq1, seq2, hsp, 4))

# test2: low score ---
print('\n--- test2 ---\n')
print('[expected]')
print(f'{None}\n')
print('[actual]')
print(local_alignment(seq1, seq2, hsp, 10))

# test3 ---
hsp = (4, 14, 8)
seq1 = 'CATSGGGGGGGAAACGG'
seq2 = 'ATATATATATTAGAGGGGGGGTAGCCGAATTAATATATATATATTATA'
print('\n--- test3 ---\n')
print('[expected]')
print("""\
target            2 TS-G-GGGGGG--AAA-C-G 16
                  0 |--|-||||||--|---|-| 20
query             0 T-AGAGGGGGGGTA--GCCG 17
""")
print('[actual]')
print(local_alignment(seq1, seq2, hsp, 4))

# test4: overhanging seq2 bounds ---
hsp = (3, 10, 7)
seq1 = 'AAAGGGGGGAAAA'
seq2 = 'ATATATATATGGGGGG'
print('\n--- test4 ---\n')
print('[expected]')
print("""\
target            0 AAA-GGGGGG  9
                  0 |---|||||| 10
query             1 A--TGGGGGG  9
""")
print('[actual]')
print(local_alignment(seq1, seq2, hsp, 4))




--- test1 ---

[expected]
target            0 GGGG 4
                  0 |||| 4
query             0 GGGG 4

[actual]


NameError: name 'local_alignment' is not defined

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 6a</h3>

<b>Challenge:</b> Write a function (or multiple) which identifies possible plastic-degrading proteins in the 'unknown' organism. 
- [ ] Input:
    - Filepath to 'query' unknown organism FASTA File (str)
    - Filepath to 'reference' PlasticDB FASTA File (str)
    - a k-mer size (int)
- [ ] Output: a list of tuples (Tuple[str, str])
- [ ] Return an empty list if no matching sequences. 

Your output should be a list of tuples, where each tuple represents a sequence match between a protein from the 'unknown' bacteria, and a protein from the database. 

For example, if the protein named <small>`WP_003141008.1`</small> from our 'unknown' bacteria is similar to the PlasticDB protein named <small>`00148||Protease||Amycolatopsis_orientalis||PLA`</small>, our output to the final question should contain the following tuple: 
<small>`('WP_003141008.1', '00148||Protease||Amycolatopsis_orientalis||PLA')`</small>

Notes:
- You are free to implement this question however you like, but you must use indexing in your solution.
- You may use functions defined in previous questions. 
- Global-scope constants have been defined for values relating to previous questions, if using these in your solution.
- A helper function has been provided to load proteins from file.

***1 mark*** is devoted to correct output. If your output is correct, an additional ***1 mark*** can be gained for quick runtime. <br>
Begin with a solution which works, then optimise later if runtime is larger than desired.

The definition of 'correct' output differs based on your implementation. <br>
We will be checking the rough number of results you return, and whether specific tuples are in your output list. <br>
You will need to decide for yourself whether your output is reasonable.


In [ ]:
# helper function to load proteins from 'unknown' organism fasta file and PlasticDB fasta file. 

def load_proteins(filepath: str):
    for seq in SeqIO.parse(filepath, "fasta"):
        yield (seq.name, str(seq.seq))


In [ ]:
# -- GRADED CELL (2 marks) - complete this cell --

import Bio.Align

MAX_SPACING     = 8      # question 3
MAX_MISMATCHES  = 6      # question 4
MIN_ALIGN_SCORE = 10     # question 5

def identify_plastic_degrading_proteins(queries_filepath: str, reference_filepath: str, k: int) -> list[Tuple]:
    # Load proteins from files
    query_proteins = list(load_proteins(queries_filepath))
    reference_proteins = list(load_proteins(reference_filepath))

    # Initialize aligner
    aligner = Bio.Align.PairwiseAligner()
    aligner.mode = 'local'
    aligner.mismatch_score = -1
    aligner.match_score = 2
    aligner.open_gap_score = -1
    aligner.extend_gap_score = -0.5
    aligner.target_end_gap_score = 0.0
    aligner.query_end_gap_score = 0.0

    results = []

    # Iterate through query and reference proteins
    for query_id, query_seq in query_proteins:
        for ref_id, ref_seq in reference_proteins:
            if len(query_seq) < k or len(ref_seq) < k:  # Ensure sequence length is at least k
                continue

            # Perform alignment
            score = aligner.score(query_seq, ref_seq)

            if score >= MIN_ALIGN_SCORE:
                results.append((query_id, ref_id))

    return results
    

ModuleNotFoundError: No module named 'Bio'

In [ ]:
# extra code cell for development if needed

In [ ]:

# ensure you have downloaded the two fasta files, created a 'data' folder, and placed the fasta files in this folder.
# the paths below are relative paths, so the 'data' folder must be located in your current working directory. 
UNKNOWN_PROTEOME_PATH = 'data/unknown.fasta'
PLASTICDB_DATABASE_PATH = 'data/PlasticDB.fasta'

identify_plastic_degrading_proteins(UNKNOWN_PROTEOME_PATH, PLASTICDB_DATABASE_PATH, 6)


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">
<h3>Question 6b</h3>

(Short Answer Question)<br>
(1 mark, max 100 words)

<div class="alert alert-info">

Describe your overall approach to Question 6a, noting any strategies to improve runtime. (0.5 marks)

Suggest an additional strategy which could be implemented to further improve runtime performance or space efficiency.  (0.5 marks)
 
</div>

<span style="color:rgb(17, 122, 121); font-family:Courier"><i><b># -- GRADED CELL (1 mark) - complete this cell --</b></i></span>

YOUR ANSWER HERE


<div style="background: rgb(255,165,0); border: solid 1px rgb(129,199,132); padding: 10px;">    
<h1>END OF ASSIGNMENT</h1>
</div>


## Submitting

Follow these steps to submit your assignment

1) Before you turn this assignment in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2) Make sure you have filled in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE"

3) **Clear all output** (in the menubar, select Kernel$\rightarrow$Restart & Clear Output)

4) Your completed notebook file containing all your answers must be turned in via LMS in `.ipynb` format.

5) You must also submit a copy of this notebook in `html` format with the output cleared (see step 3).


Your submission should include **only two** files with names formatted as: **Assignment_1.ipynb** and **Assignment_1.html**